In [2]:
import torch
import torch.nn.functional as F

In [3]:
words = open('names.txt','r').read().splitlines()

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
#print(itos)

In [6]:
#prep data
window = 3

def build_dataset(words):
    xs = []
    ys = []
    for w in words:
        context = [0]*window
        for c in w + '.':
            ix = stoi[c]
            xs.append(context)
            ys.append(ix)
            context = context[1:]+[ix]
    X = torch.tensor(xs)
    Y = torch.tensor(ys)
    return X,Y
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr, Ytr =build_dataset(words[:n1])
Xval, Yval = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [17]:
Xtr.shape

torch.Size([182625, 3])

In [7]:
#initialize params
g = torch.Generator().manual_seed(2147483647)
emb_dim = 10
C = torch.randn((27, emb_dim), generator=g)
W1 = torch.randn((window*emb_dim, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [14]:
#FP, BP
for i in range(200000):
    ix = torch.randint(0, Xtr.shape[0], (32,))
    X = Xtr[ix]
    Y = Ytr[ix]
    emb = C[X] # (32, 3, 10)
    h = torch.tanh(emb.view(-1, window*emb_dim) @ W1 + b1) 
    logit = h @ W2 + b2
    loss = F.cross_entropy(logit, Y)
    for p in parameters:
        p.grad = None
    loss.backward()

    lr = 0.1 if i < 10000 else 0.01
    for p in parameters:
        p.data += -lr *p.grad
print(loss.item())

2.4972496032714844


In [15]:
g = torch.Generator().manual_seed(2147483647 + 10)
for _ in range(20):
    oup = []
    context = [0]*window
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        oup.append(ix)
        context = context[1:] + [ix]
        #print(ix)
        if ix == 0:
            #print('done')
            break
    print(''.join([itos[i] for i in oup]))


carmah.
amerilli.
kimli.
reviyah.
cassie.
mahnen.
delyah.
jareei.
nelynna.
chaiiv.
kaleigh.
ham.
jock.
quinn.
sulin.
alian.
quinatha.
dearyxi.
jaxenni.
say.


In [18]:
#create classes
class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in ** 0.5
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight]  + ([] if self.bias is None else [self.bias])

class BatchNorm1d:
    def __init__(self, dim, eps = 1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    def __call__(self, x):
        if self.training:
            mean = x.mean(0, keepdim=True)
            var = x.var(0, keepdim=True)
        else:
            mean = self.running_mean
            var = self.running_var
        self.out = self.gamma * ((x - mean) / torch.sqrt(var + self.eps)) + self.beta
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var
        return self.out    
    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []

In [22]:
#initialize
n_emb = 10
n_hidden = 100
g = torch.Generator().manual_seed(2147483647)
vocab_size = len(stoi)

C = torch.randn((vocab_size, n_emb), generator=g)
layers = [
    Linear(n_emb*window, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(    n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(    n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(    n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(    n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(    n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),]

with torch.no_grad():
    layers[-1].gamma *= 0.1 #make it less confident
    for layer   in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True


47024


In [23]:
#train
max_steps = 2000
batch_size = 32
lossi = []
ud = []

for i in range(max_steps):
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]

    #forward
    x = C[Xb].view(-1, n_emb * window) # is this the same?
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)

      # backward pass
    for layer in layers:
        layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph

    for p in parameters:
        p.grad = None
    loss.backward()

    lr = 0.1 if i<150000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
     # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
        lossi.append(loss.log10().item())
        with torch.no_grad():
            ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

    if i >= 1000:
        break # AFTER_DEBUG: would take out obviously to run full optimization

In [ ]:
#test